In [1]:
import numpy as np
import pandas as pd
import sklearn
import warnings
warnings.filterwarnings('ignore')
from numpy.testing import assert_array_equal, assert_array_almost_equal, assert_equal, assert_almost_equal
from pandas.testing import assert_frame_equal

In [2]:
train = pd.read_csv("resources/train.csv")
test = pd.read_csv("resources/test.csv")
sample_submission = pd.read_csv("resources/sample_submission.csv")

# SLIDE (1) Удаление Nan

Серия задач в данном модуле объединена в одну [большую задачу по предсказанию данных](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/overview). В ходе выполнения модуля мы будем разбирать определенные техники, которые нужны для ее решения. Настоятельно рекомендуем выполнить все шаги по порядку, тогда в конце вы получите решение большой реальной задачи по МЛ.

Нам даны [данные](https://yadi.sk/d/tcElz6cqSNpPqA) о домах выставленных на продажу. Нам необходимо решить задачу регрессии и предсказать цену продажи дома для $X_{test}$ по данным $X_{train}$ и $y_{train}$. В нашем случае  $y_{train}$ - это столбик `SalePrice`, $X_{train}$ - все остальные столбики.

На вход подается 2 считанных датафрейма **df_train**, **df_test** из файлов без изменений. 

Начальная подготовка:
 * Разделить **df_train** на **X_train**(`pd.Dataframe`) и **y_train**(`pd.Series`).
 * Сконкатенировать **X_train** и **df_test** в **df** по вертикали (можно ориентироваться по столбику `Id` они как раз идут по-порядку). Не забудьте обновить индекс!

Задачи:
 * Заменить в **df** все Nan-ы в категориальных признаках (`object`) на строку `missing`
 * Заменить в **df** все Nan-ы в числовых признаках на 0.

Вернуть из функции измененный **df**.

In [23]:
import pandas as pd

def del_nan(df_train: pd.DataFrame, df_test: pd.DataFrame) -> pd.DataFrame:
    y_train = df_train['SalePrice']
    X_train = df_train.drop(['SalePrice'], axis='columns')
    df = pd.concat([X_train, df_test], axis=0)
    print(df)
    df = df.reset_index(drop=True)
    df = df.select_dtypes(include='object').fillna('missing')
    df = df.fillna(0)
    print(df)
    return df

del_nan(train, test)

        Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0        1          60       RL         65.0     8450   Pave   NaN      Reg   
1        2          20       RL         80.0     9600   Pave   NaN      Reg   
2        3          60       RL         68.0    11250   Pave   NaN      IR1   
3        4          70       RL         60.0     9550   Pave   NaN      IR1   
4        5          60       RL         84.0    14260   Pave   NaN      IR1   
5        6          50       RL         85.0    14115   Pave   NaN      IR1   
6        7          20       RL         75.0    10084   Pave   NaN      Reg   
7        8          60       RL          NaN    10382   Pave   NaN      IR1   
8        9          50       RM         51.0     6120   Pave   NaN      Reg   
9       10         190       RL         50.0     7420   Pave   NaN      Reg   
10      11          20       RL         70.0    11200   Pave   NaN      Reg   
11      12          60       RL         85.0    1192

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,RL,Pave,missing,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,Attchd,RFn,TA,TA,Y,missing,missing,missing,WD,Normal
1,RL,Pave,missing,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,...,Attchd,RFn,TA,TA,Y,missing,missing,missing,WD,Normal
2,RL,Pave,missing,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,Attchd,RFn,TA,TA,Y,missing,missing,missing,WD,Normal
3,RL,Pave,missing,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,...,Detchd,Unf,TA,TA,Y,missing,missing,missing,WD,Abnorml
4,RL,Pave,missing,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,...,Attchd,RFn,TA,TA,Y,missing,missing,missing,WD,Normal
5,RL,Pave,missing,IR1,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,...,Attchd,Unf,TA,TA,Y,missing,MnPrv,Shed,WD,Normal
6,RL,Pave,missing,Reg,Lvl,AllPub,Inside,Gtl,Somerst,Norm,...,Attchd,RFn,TA,TA,Y,missing,missing,missing,WD,Normal
7,RL,Pave,missing,IR1,Lvl,AllPub,Corner,Gtl,NWAmes,PosN,...,Attchd,RFn,TA,TA,Y,missing,missing,Shed,WD,Normal
8,RM,Pave,missing,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Artery,...,Detchd,Unf,Fa,TA,Y,missing,missing,missing,WD,Abnorml
9,RL,Pave,missing,Reg,Lvl,AllPub,Corner,Gtl,BrkSide,Artery,...,Attchd,RFn,Gd,TA,Y,missing,missing,missing,WD,Normal


# SLIDE (2) Порядковые категории

Вам на вход приходит **df** из предыдущей задачи.

Если внимательно изучить файл `data_description` можно понять, что многие категориальные признаки - порядковые (упорядоченное множество). Значит их можно перевести в осмысленные числа. Значит тут можно воспользоваться `LabelEncoding`.

Ваша задача: заменить в **df** категориальные признаки на числовые, для порядковых признаков.

На выходе возвращаем измененный **df**.

Чтобы слегка упростить вам жизнь, вот вам готовые словари для перевода. Однако к каким столбцам их применять - вы должны выяснить сами, изучив файл `data_description`. Каждый маппинг используется хотя бы 1 раз, а некоторые и не по одному разу.

```python
{'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'missing':0}
{'Gd':4, 'Av': 3, 'Mn': 2, 'No': 1, 'missing': 0}
{'GLQ': 6, 'ALQ': 5, 'BLQ': 4, 'Rec': 3, 'LwQ': 2, 'Unf': 1, 'missing': 0}
{'Typ': 8, 'Min1': 7, 'Min2': 6, 'Mod': 5, 'Maj1': 4, 'Maj2': 3, 'Sev': 2, 'Sal': 1, 'missing': 0}
{'Fin': 3, 'RFn': 2, 'Unf': 1, 'missing': 0}
{'GdPrv': 4, 'MnPrv': 3, 'GdWo': 2, 'MnWw': 1, 'missing': 0}
{'Reg': 4, 'IR1': 3, 'IR2': 2, 'IR3': 1, 'missing': 0}
{'Lvl': 4, 'Bnk': 3, 'HLS':2,'Low':1, 'missing': 0}
{'AllPub':4, 'NoSewr':3, 'NoSeWa':2, 'ELO':1, 'missing':0}
{'Gtl':3, 'Mod':2, 'Sev':1, 'missing':0}
{'SBrkr':5, 'FuseA':4, 'FuseF':3, 'FuseP':2, 'Mix':1, 'missing':0}
{'Y':3, 'P':2, 'N':1, 'missing':0}
{'Y':1, 'N':0, 'missing':0} #тут нет ошибки, все так и задумано:)
```

Подсказка: после перевода у вас должно остаться 21 категориальных признака

In [105]:
import pandas as pd

def cat_to_num(df: pd.DataFrame) -> pd.DataFrame:
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
    pass

# SLIDE (1) One hot encoding

Теперь разберемся с непорядковыми категориальными признаками.

Для начала заметим признак `MSSubClass`, у которого тип `int64`, но если посмотреть в описание `data_description` можно понять, что это - категориальный признак. 
 * Измените тип признака `MSSubClass` с `int64` на `object`

Теперь можно сделать `One hot encoding`:
 * Найдите все колонки с категориальными признаками и составьте из них отдельный **df_oh** `pd.DataFrame` (индекс сохранить прежний)
 * Применить к полученному фрейму **df_oh** функцию [`pd.get_dummies`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html) (Реализует `One Hot Encoding`)
 * Удалить категориальные колонки из **df** и добавить справа к **df** фрейм с `One Hot Encoding`
 
Вернуть из функции **df**

Подсказка: итого должно получится 241 колонка из них 183 пришли из `One hot encoding` фрейма

In [182]:
import pandas as pd

def one_hot(df: pd.DataFrame) -> pd.DataFrame:
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
    pass

# SLIDE (2) Некоррелирующие признаки 

Мы разобрались с категориальными признаками, теперь разберемся с числовыми.
Для числовых признаков можно посчитать корреляцию с правильным ответом. Если признаки слабо коррелируют, то они нам не нужны. Например колонка `Id` явно никак не влияет на стоимость дома.

Вам на вход передается изначальный **df_train** и **df** полученный из предыдущей задачи.

Ваша задача: 
 * найти корреляцию всех **числовых** признаков **df_train** с признаком `SalePrice` с помощью `pd.corr`
 * если абсолютное значение корреляции признака с `SalePrice` меньше $0.05$ - удалите этот признак из **df**

Верните измененный **df** и столбец корреляции признаков с признаком `SalePrice` упорядоченный по убыванию. Начало столбца корреляции выглядит следующим образом:

|               |SalePrice |
|---------------|----------|
|**SalePrice**  |1.000000  |
|**OverallQual**|0.790982  |
|**GrLivArea**  |0.708624  |
|**GarageCars** |0.640409  |
|**GarageArea** |0.623431  |
|**TotalBsmtSF**|0.613581  |
|**1stFlrSF**   |0.605852  |
|**FullBath**   |0.560664  |

Всего должно получиться 37 числовых признаков.

Подсказка: как мы помним столбец`MSSubClass` - категориальный, уберите его из **df_train** при рассмотрении корреляции.

Подсказка: всего придется удалить 8 признаков (включая `Id`). В **df** останется 233 признака.

In [242]:
def correlation(df: pd.DataFrame, df_train: pd.DataFrame) -> (pd.DataFrame, pd.DataFrame):
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
    pass

# SLIDE (1) Feature Engineering

Давайте нагенерируем несколько фич во входном фрейме **df**:
 * `TotalArea` = `TotalBsmtSF` + `1stFlrSF` + `2ndFlrSF` + `GrLivArea` + `GarageArea`
 * `YearAverage` = (`YearRemodAdd` + `YearBuilt`) / 2
 * `LiveAreaQual` = `OverallQual` * `GrLivArea`

На выход отправьте **df** c тремя новымим столбиками. столбцы должны идти в том же порядки что указаны в списке в хвосте **df**.

In [266]:
import pandas as pd

def feature_en(df: pd.DataFrame) -> pd.DataFrame:
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
    pass

# SLIDE (1) Scaling с выбросами

У стандартного и нормального масштабирования есть одна проблема: она учитывает все признаки, даже те, которые изначально некорректны (шум, выбросы). Чтобы избавитьться от шумов и выбросов и корректно масштабировать выборку необходимо использовать [RobustScaling]((https://scikit-learn.org/0.18/auto_examples/preprocessing/plot_robust_scaling.html).

Ваша задача - отмасштабировать полученный фрейм с помощью `RobustScaler`. И вернуть отмасштабированный массив (да, скалирование возвращает массив, а не DataFrame).

In [355]:
import numpy as np
import pandas as pd

def scaling(df: pd.DataFrame) -> np.array:
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
    pass

# SLIDE (2) Смешанная модель

Отлично, теперь мы готовы обучать модель! Осталось изучить последний интересный трюк - смешанные модели.

Возьмем [2 регрессии](https://towardsdatascience.com/ridge-and-lasso-regression-a-complete-guide-with-python-scikit-learn-e20e34bcbf0b):
 * Ridge
 * Lasso
 
Найдем оптимальные значения $\alpha$ для обеих регрессий с помощью GridSearch.

Теперь отправим обе модели с наилучшими параметрами в класс указанный снизу. Это класс смешения моделей. В нем параметр $\beta \in [0,1]$ - это коэффициент, с которым берется ответ одного классификаторв, а ответ второго - с коэффициентом $(1 - \beta)$. Такая техника нередко позволяет добиться лучших результатов, чем одна модель.

Теперь найдем наилучшее $\beta$ для смешенной модели также с помощью GridSearch. Осталось получить **y_pred** с помощью наилучшей смешанной модели.

На вход вы получаете **X_scaled** из предыдущей задачи и **df_train** начальный. Мы подготовили за вас **X_train**, **y_train** и **X_test**. 

В задаче необходимо минимизировать метрику `neg_mean_squared_log_error`. Для удобства мы возьмем `np.log1p(y_train)` и будем минимизировать метрику `neg_mean_squared_error`. Эту метрику необходимо минимизировать у всех 3-х GridSearch.

На выход отправьте GridSearch объект смешанной модели, а также результат **y_test**. (Не забудьте его проэкспоненциировать).

Мы выдаем вам ориентировачные параметры для каждого GridSearch. Вы можете увеличить перебор, чтобы получить лучшую модель.
```python
params_ridge = {'alpha': np.arange(1, 20)}
params_lasso = {'alpha': np.logspace(-4, 3, num=8, base=10)}
params_blend = {'beta': np.linspace(0, 1, 11)}
```

Первые 2 GridSearch **не нужно** писать в функции: они могут работать достаточно долго и превысят лимит работы задачи на сервере. Найдите у себя локально наилучшие параметры и уже с этими параметрами создайте смешанную модель внутри функции. 

Также не нужно сильно увеличивать перебор для $\beta$ - того, что есть, более чем достаточно.

P.S. Осталось сохранить файл с **y_test** и отправить его в [соревнование](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/submit). Улучшайте свои результы пробуя другие модели и другие параметры.


In [373]:
from sklearn.base import BaseEstimator
from sklearn.model_selection import GridSearchCV
class BlendRegressor(BaseEstimator): # предок класса классификаторов, чтобы можно было засунуть в GridSearch
    def __init__(self, clf1, clf2, beta=0.5):
        self.clf1 = clf1 
        self.clf2 = clf2
        self.beta = beta #параметр смешивания

    def fit(self, X, y): #обучаем классификатор
        self.X_ = X, self.y_ = y 
        self.clf1.fit(X, y), self.clf2.fit(X, y)
        return self

    def predict(self, X): #возвращаем значения 
        return self.clf1.predict(X) * self.beta + self.clf2.predict(X) * (1 - self.beta)

    
def learning(X_scaled: np.array, df_train: pd.DataFrame) -> (GridSearchCV, np.array):
    X_train = X_scaled[0: len(df_train),]
    X_test  = X_scaled[len(df_train): len(X_scaled)]
    y_train = np.log1p(df_train['SalePrice'])
    
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
    
    return blend_gs, np.exp(y_test) - 1